In [17]:
import pandas as pd
import numpy as np
from pathlib import Path
import gc
from sklearn.impute import KNNImputer

def standardize_column_names(columns):
    """
    Uniformiza os nomes das colunas removendo a letra do meio entre os underscores.
    """
    standardized = []
    for col in columns:
        parts = col.split('_')
        if len(parts) == 3 and len(parts[1]) == 1:  # Se tem formato XXX_Y_ZZZ
            standardized.append(f"{parts[0]}_{parts[2]}")
        else:
            standardized.append(col)
    return standardized

def load_and_preprocess_nhanes_2013_2014(file_path):
    """
    Carrega e pré-processa os dados do NHANES 2013-2014
    """
    try:
        # Carregar dados
        df = pd.read_csv(file_path)
        print(f"Dados 2013-2014 carregados. Shape inicial: {df.shape}")
        
        # Remover colunas específicas não desejadas
        columns_to_remove = [
            'DXXFEM_H_DXXINBMD', 'DXXSPN_H_DXXOSBMD.1', 'DXXFEM_H_DXXWDBMD',
            'DXXSPN_H_DXXL1BMD', 'DXXFEM_H_DXXTRBMD', 'DXXSPN_H_DXXL2BMD',
            'DXXSPN_H_DXXL3BMD', 'DXXSPN_H_DXXL4BMD'
        ]
        df = df.drop(columns=columns_to_remove, errors='ignore')
        
        # Uniformizar nomes das colunas
        df.columns = standardize_column_names(df.columns)
        
        # Remover colunas duplicadas
        df = df.loc[:, ~df.columns.duplicated()]
        print(f"Shape após processamento: {df.shape}")
        
        return df
        
    except Exception as e:
        print(f"Erro ao carregar dados 2013-2014: {str(e)}")
        return None

def load_nhanes_cycle(directory_path, reference_columns):
    """
    Carrega e processa dados de um ciclo do NHANES
    """
    directory = Path(directory_path)
    if not directory.exists():
        print(f"Diretório {directory} não existe.")
        return None
    
    try:
        dataframes = []
        
        # Primeira passagem: carregar todos os arquivos XPT
        for file_path in directory.glob('*.XPT'):
            try:
                df = pd.read_sas(file_path, format='xport')
                file_name = file_path.stem
                
                if 'SEQN' in df.columns:
                    # Renomear colunas com prefixo do nome do arquivo
                    rename_dict = {col: f"{file_name}_{col}" if col != 'SEQN' else col 
                                 for col in df.columns}
                    df = df.rename(columns=rename_dict)
                    
                    # Garantir que SEQN seja único
                    df = df.sort_values('SEQN').groupby('SEQN').first().reset_index()
                    
                    dataframes.append(df)
                    print(f"Arquivo válido: {file_name}, Shape: {df.shape}")
                    
            except Exception as e:
                print(f"Erro ao ler {file_path}: {str(e)}")
        
        if not dataframes:
            print("Nenhum arquivo XPT válido encontrado.")
            return None
            
        # Realizar o merge de todos os DataFrames
        print("\nIniciando processo de merge...")
        df_merged = dataframes[0]
        for i, df in enumerate(dataframes[1:], 1):
            print(f"\nMergindo DataFrame {i} de {len(dataframes)-1}")
            df_merged = pd.merge(df_merged, df, on='SEQN', how='outer')
            gc.collect()
        
        # Limpar memória
        del dataframes
        gc.collect()
        
        # Uniformizar nomes das colunas
        df_merged.columns = standardize_column_names(df_merged.columns)
        
        # Selecionar apenas as colunas de interesse
        common_columns = set(df_merged.columns) & set(reference_columns)
        df_merged = df_merged[list(common_columns)]
        
        # Remover linhas com dados faltantes nos valores de BMD
        bmd_columns = ['DXXFEM_DXXNKBMD', 'DXXFEM_DXXOFBMD', 'DXXSPN_DXXOSBMD']
        df_merged = df_merged.dropna(subset=bmd_columns, how='all')
        
        # Filtrar para idade >= 40 anos
        print(f"\nShape antes do filtro de idade: {df_merged.shape}")
        df_merged = df_merged[df_merged['DEMO_RIDAGEYR'] >= 50]
        print(f"Shape após filtro de idade (>=50 anos): {df_merged.shape}")
        
        print(f"\nDistribuição de idade após filtro:")
        print(df_merged['DEMO_RIDAGEYR'].describe())
        
        print(f"\nShape final após processamento: {df_merged.shape}")
        return df_merged
        
    except Exception as e:
        print(f"Erro ao processar ciclo NHANES: {str(e)}")
        return None

def calculate_t_scores(df):
    """
    Calcula T-scores para diferentes regiões ósseas usando valores de referência atualizados
    """
    # Definir valores de referência atualizados
    reference_values = {
        'fn': {  # Femoral Neck
            'white': {'female': (0.875, 0.126), 'male': (0.929, 0.133)},
            'black': {'female': (0.981, 0.156), 'male': (1.037, 0.160)},
            'hispanic': {'female': (0.869, 0.114), 'male': (0.968, 0.131)}
        },
        'tf': {  # Total Femur
            'white': {'female': (0.960, 0.123), 'male': (1.048, 0.136)},
            'black': {'female': (1.044, 0.136), 'male': (1.148, 0.164)},
            'hispanic': {'female': (0.954, 0.120), 'male': (1.083, 0.136)}
        },
        'sp': {  # Spine
            'white': {'female': (1.046, 0.110), 'male': (1.047, 0.118)},
            'black': {'female': (1.118, 0.129), 'male': (1.101, 0.120)},
            'hispanic': {'female': (1.019, 0.108), 'male': (1.041, 0.112)}
        }
    }
    
    def get_reference_values(gender, race, site):
        if race in [1, 2]:  # Hispanic
            ethnicity = 'hispanic'
        elif race == 4:  # Black
            ethnicity = 'black'
        else:  # White and others
            ethnicity = 'white'
        
        gender_str = 'female' if gender == 2 else 'male'
        return reference_values[site][ethnicity][gender_str]
    
    def calculate_single_t_score(bmd, mean, sd):
        return (bmd - mean) / sd if pd.notnull(bmd) else np.nan
    
    # Calcular T-scores para cada região
    sites = {
        'fn': 'DXXFEM_DXXNKBMD',
        'tf': 'DXXFEM_DXXOFBMD',
        'sp': 'DXXSPN_DXXOSBMD'
    }
    
    for site, column in sites.items():
        df[f'T_score_{site}'] = df.apply(
            lambda row: calculate_single_t_score(
                row[column],
                *get_reference_values(row['DEMO_RIAGENDR'], row['DEMO_RIDRETH1'], site)
            ),
            axis=1
        )
    
    print(f"T-scores calculados para {df.shape[0]} registros")
    return df

def create_target_variable(df):
    """ 
    Cria variável target baseada nos T-scores e outros critérios:
    - T-score ≤ -2.5 em qualquer sítio
    - Percentis calculados separadamente para homens e mulheres
    """
    try:
        # Separar por género
        df_male = df[df['DEMO_RIAGENDR'] == 1].copy()
        df_female = df[df['DEMO_RIAGENDR'] == 2].copy()
        
        # Função para calcular percentis
        def calculate_percentiles(data):
            data["BMI"]=data['BMX_BMXWT']/(data['BMX_BMXHT']*data['BMX_BMXHT'])
            data['BMI_percentil'] = pd.qcut(data['BMI'], q=15, labels=False, duplicates='drop')
            data['RIDAGEYR_percentil'] = pd.qcut(data['DEMO_RIDAGEYR'], q=15, labels=False, duplicates='drop')
            data['BMXWT_percentil'] = pd.qcut(data['BMX_BMXWT'], q=15, labels=False)
            data['BMXHT_percentil'] = pd.qcut(data['BMX_BMXHT'], q=15, labels=False)
            data['BMXARMC_percentil'] = pd.qcut(data['BMX_BMXARMC'], q=15, labels=False)
            return data
        
        # Calcular percentis separadamente
        df_male = calculate_percentiles(df_male)
        df_female = calculate_percentiles(df_female)
        
        # Recombinar os dados
        df = pd.concat([df_male, df_female])
        
        def categorize_bone_health(row):
            t_scores = [row[f'T_score_{site}'] for site in ['fn', 'tf', 'sp']]
            t_scores = [score for score in t_scores if pd.notnull(score)]
            
            if not t_scores:
                return np.nan
            
            min_t_score = min(t_scores)
            
            # Inicializa o critério antropométrico como False
            anthropometric_criteria = False
            
            # Critério de peso aplicado para todos
            if row.get('BMXWT_percentil') == 0 or row.get('BMXARMC_percentil') == 0:
                anthropometric_criteria = True
            
            # Critério de idade aplicado apenas para mulheres
            if row.get('DEMO_RIAGENDR') == 2 and row.get('RIDAGEYR_percentil') == 13:
                anthropometric_criteria = True

            if min_t_score <= -2.5 or anthropometric_criteria:
                return 1
            return 0

        # Criar variável target
        df['target'] = df.apply(categorize_bone_health, axis=1)
        
        # Análise dos critérios de classificação
        def analyze_classification_criteria(df):
            total_valid = df['target'].notna().sum()
            osteo = (df['target'] == 1).sum()
            
            criteria_counts = {
                'T-score ≤ -2.5': sum((df[[f'T_score_{site}' for site in ['fn', 'tf', 'sp']]].min(axis=1) <= -2.5) & (df['target'] == 1)),
                'All T-scores ≤ -1.0': sum(df[[f'T_score_{site}' for site in ['fn', 'tf', 'sp']]].apply(lambda x: all(score <= -1.0 for score in x), axis=1) & (df['target'] == 1)),
                'Age (highest percentile)': sum((df['RIDAGEYR_percentil'] == 19) & (df['target'] == 1)),
                'Weight (lowest percentile)': sum((df['BMXWT_percentil'] == 0) & (df['target'] == 1)),
                'Arm circumference (lowest percentile)': sum((df['BMXARMC_percentil'] == 0) & (df['target'] == 1))
            }
            
            print("\nCritérios de classificação para casos positivos:")
            for criterion, count in criteria_counts.items():
                print(f"{criterion}: {count} casos ({count/osteo*100:.1f}% dos positivos)")
        
        # Análises separadas por género
        for gender, label in [(1, 'Homens'), (2, 'Mulheres')]:
            gender_df = df[df['DEMO_RIAGENDR'] == gender]
            total_valid = gender_df['target'].notna().sum()
            normal = (gender_df['target'] == 0).sum()
            osteo = (gender_df['target'] == 1).sum()
            
            print(f"\nDistribuição da variável target para {label}:")
            print(f"Normal (0): {normal} ({normal/total_valid*100:.1f}%)")
            print(f"Osteoporose (1): {osteo} ({osteo/total_valid*100:.1f}%)")
            
            print(f"\nPercentis para {label}:")
            for col in ['BMX_BMXWT', 'BMX_BMXHT', 'BMX_BMXARMC']:
                print(f"\n{col}:")
                print(gender_df[col].describe(percentiles=[.05, .10, .25, .50, .75, .90, .95]))
            
            # Análise dos critérios de classificação para cada género
            print(f"\nAnálise dos critérios para {label}:")
            analyze_classification_criteria(gender_df)
        
        return df
        
    except Exception as e:
        print(f"Erro ao criar variável target: {str(e)}")
        return None

def combine_and_impute_data(dfs):
    """
    Combina diferentes ciclos do NHANES e imputa valores faltantes,
    criando flags para valores imputados
    """
    try:
        # Combinar DataFrames
        df_combined = pd.concat(dfs, ignore_index=True)
        print(f"\nShape após combinação dos ciclos: {df_combined.shape}")
        
        # Separar variáveis numéricas e categóricas
        numeric_cols = df_combined.select_dtypes(include=['float64', 'int']).columns
        categorical_cols = df_combined.select_dtypes(include=['object']).columns
        
        # Criar dicionário para armazenar flags de imputação
        imputation_flags = {}
        
        # Para cada coluna numérica, criar uma flag de valores faltantes
        for col in numeric_cols:
            imputation_flags[f"{col}_imputed"] = df_combined[col].isnull()
        
        # Imputar valores numéricos com KNN
        print("\nIniciando imputação de valores numéricos...")
        imputer = KNNImputer(n_neighbors=3)
        df_combined[numeric_cols] = imputer.fit_transform(df_combined[numeric_cols])
        
        # Imputar valores categóricos com moda e criar flags
        print("\nIniciando imputação de valores categóricos...")
        for col in categorical_cols:
            imputation_flags[f"{col}_imputed"] = df_combined[col].isnull()
            df_combined[col].fillna(df_combined[col].mode()[0], inplace=True)
        
        # Adicionar flags ao DataFrame
        for flag_name, flag_values in imputation_flags.items():
            df_combined[flag_name] = flag_values.astype(int)
        
        # Análise da imputação
        print("\nAnálise da Imputação:")
        print("-" * 50)
        
        for col in numeric_cols:
            flag_col = f"{col}_imputed"
            if flag_col in df_combined.columns:
                total_imputed = df_combined[flag_col].sum()
                imputed_class_1 = df_combined[df_combined['target'] == 1][flag_col].sum()
                imputed_class_0 = df_combined[df_combined['target'] == 0][flag_col].sum()
                
                print(f"\nColuna: {col}")
                print(f"Total de valores imputados: {total_imputed}")
                print(f"Valores imputados na classe 0: {imputed_class_0}")
                print(f"Valores imputados na classe 1: {imputed_class_1}")
                print(f"Porcentagem na classe 1: {(imputed_class_1/total_imputed*100 if total_imputed > 0 else 0):.2f}%")
                
                # Análise estatística dos valores imputados vs. não imputados
                print("\nEstatísticas dos valores:")
                print("Originais:")
                print(df_combined[df_combined[flag_col] == 0][col].describe().round(2))
                print("\nImputados:")
                print(df_combined[df_combined[flag_col] == 1][col].describe().round(2))
        
        print("\nImputação concluída com sucesso!")
        return df_combined
        
    except Exception as e:
        print(f"Erro ao combinar e imputar dados: {str(e)}")
        return None

# Exemplo de uso:
# df_final = combine_and_impute_data([df1, df2, df3])

In [18]:
import pandas as pd
import numpy as np
from pathlib import Path
import gc

# Definir caminhos dos arquivos
BASE_PATH = Path('/Users/filipecarvalho/Documents/data_science_projects/BMD/paper/Computers_in_Biology_and_Medicine/NHANES')
PATHS = {
    '2013-2014': BASE_PATH / 'variaveis_selecionadas.csv',
    '2009-2010': BASE_PATH / 'NHANES_2009_2010',
    '2007-2008': BASE_PATH / 'NHANES_2007_2008'
}

# Carregar e processar dados do NHANES 2013-2014 (dados de referência)
print("\nCarregando dados NHANES 2013-2014...")
df_2013_2014 = load_and_preprocess_nhanes_2013_2014(PATHS['2013-2014'])

if df_2013_2014 is not None:
    # Obter lista de colunas de referência após processamento
    reference_columns = standardize_column_names(df_2013_2014.columns)
    print(f"\nNúmero de colunas de referência: {len(reference_columns)}")
    
    # Carregar e processar ciclos 2009-2010 e 2007-2008
    cycle_dfs = {'2013-2014': df_2013_2014}
    
    for cycle in ['2009-2010', '2007-2008']:
        print(f"\nCarregando dados NHANES {cycle}...")
        cycle_dfs[cycle] = load_nhanes_cycle(PATHS[cycle], reference_columns)
    
    # Verificar se todos os DataFrames foram carregados corretamente
    if all(df is not None for df in cycle_dfs.values()):
        print("\nTodos os ciclos foram carregados com sucesso")
        
        # Calcular T-scores para cada ciclo
        dfs_with_t_scores = {}
        for cycle, df in cycle_dfs.items():
            print(f"\nCalculando T-scores para ciclo {cycle}")
            dfs_with_t_scores[cycle] = calculate_t_scores(df)
        
        # Criar target variable para cada ciclo
        dfs_with_target = {}
        for cycle, df in dfs_with_t_scores.items():
            print(f"\nCriando variável target para ciclo {cycle}")
            df_with_target = create_target_variable(df)  # Guarda o resultado em uma nova variável
            
            if df_with_target is not None:  # Verifica se a criação da target foi bem sucedida
                dfs_with_target[cycle] = df_with_target
                
                # Mostrar informações sobre o ciclo
                print(f"\nInformações do ciclo {cycle}:")
                print(f"Shape: {df_with_target.shape}")
                if 'target' in df_with_target.columns:  # Verifica se a coluna target existe
                    print("Distribuição target:")
                    print(df_with_target['target'].value_counts(normalize=True).round(3) * 100)
                else:
                    print("ERRO: Coluna target não foi criada!")
            else:
                print(f"ERRO: Falha ao criar target para ciclo {cycle}")
        
        # Combinar todos os ciclos e imputar valores faltantes
        print("\nCombinando todos os ciclos...")  
        df_final = combine_and_impute_data(list(dfs_with_target.values()))
        df_final["BMXWT_percentil"] = df_final["BMXWT_percentil"].round(0)
        df_final["BMXARMC_percentil"] = df_final["BMXARMC_percentil"].round(0)
        df_final["ALQ_ALQ101"] = df_final["ALQ_ALQ101"].round(0)
        df_final["OCQ_OCD150"] = df_final["OCQ_OCD150"].round(0)
        df_final["PAQ_PAQ635"] = df_final["PAQ_PAQ635"].round(0)
        df_final["PAQ_PAQ650"] = df_final["PAQ_PAQ650"].round(0)
        df_final["WHQ_WHQ040"] = df_final["WHQ_WHQ040"].round(0)
        df_final["INQ_INQ030"] = df_final["INQ_INQ030"].round(0)
        df_final["INQ_INQ020"] = df_final["INQ_INQ020"].round(0)
        df_final["DIQ_DIQ180"] = df_final["DIQ_DIQ180"].round(0)
        df_final["OSQ_OSQ060"] = df_final["OSQ_OSQ060"].round(0)
        df_final["MCQ_MCQ080"] = df_final["MCQ_MCQ080"].round(0)
        df_final["WHQ_WHD050"] = df_final["WHQ_WHD050"].round(0)
        df_final["WHQ_WHQ030"] = df_final["WHQ_WHQ030"].round(0)
        df_final["BMI_percentil"] = df_final["BMI_percentil"].round(0)
        df_final["BMXWT_percentil"] = df_final["BMXWT_percentil"].astype(int)
        df_final["BMXARMC_percentil"] = df_final["BMXARMC_percentil"].astype(int)
        df_final["ALQ_ALQ101"] = df_final["ALQ_ALQ101"].astype(int)
        df_final["BPX_BPACSZ"] = df_final["BPX_BPACSZ"].round(0)
        df_final["BPX_BPACSZ"] = df_final["BPX_BPACSZ"].astype(int)
        df_final["OCQ_OCD150"] = df_final["OCQ_OCD150"].astype(int)
        df_final["WHQ_WHQ040"] = df_final["WHQ_WHQ040"].astype(int)
        df_final["INQ_INQ030"] = df_final["INQ_INQ030"].astype(int)
        df_final["INQ_INQ020"] = df_final["INQ_INQ020"].astype(int)
        df_final["DIQ_DIQ180"] = df_final["DIQ_DIQ180"].astype(int)
        df_final["OSQ_OSQ060"] = df_final["OSQ_OSQ060"].astype(int)
        df_final["MCQ_MCQ080"] = df_final["MCQ_MCQ080"].astype(int)
        df_final["WHQ_WHD050"] = df_final["WHQ_WHD050"].astype(int)
        df_final["WHQ_WHQ030"] = df_final["WHQ_WHQ030"].astype(int)
        if df_final is not None:
            print("\nProcessamento concluído com sucesso!")
            print(f"Shape final do DataFrame: {df_final.shape}")
            print("\nDistribuição final da variável target:")
            print(df_final['target'].value_counts(normalize=True).round(3) * 100)
            
            # Opcionalmente, salvar o DataFrame final
            output_path = BASE_PATH / 'nhanes_combined_processed.csv'
            df_final.to_csv(output_path, index=False)
            print(f"\nDataFrame final salvo em: {output_path}")
        else:
            print("\nErro: Falha ao combinar os ciclos")
    else:
        print("\nErro: Nem todos os ciclos foram carregados corretamente")
else:
    print("\nErro: Falha ao carregar dados de referência 2013-2014")


Carregando dados NHANES 2013-2014...
Dados 2013-2014 carregados. Shape inicial: (2874, 102)
Shape após processamento: (2874, 94)

Número de colunas de referência: 94

Carregando dados NHANES 2009-2010...
Arquivo válido: APOB_F, Shape: (3581, 4)
Arquivo válido: UHM_F, Shape: (2941, 27)
Arquivo válido: OCQ_F, Shape: (6889, 24)
Arquivo válido: OHXDEN_F, Shape: (8189, 39)
Arquivo válido: WHQ_F, Shape: (6889, 63)
Arquivo válido: HEPB_S_F, Shape: (9522, 2)
Arquivo válido: BHQ_F, Shape: (6059, 12)
Arquivo válido: MMRV_F, Shape: (5652, 5)
Arquivo válido: VOCWB_F, Shape: (3745, 72)
Arquivo válido: BPQ_F, Shape: (6889, 21)
Arquivo válido: TRIGLY_F, Shape: (3581, 6)
Arquivo válido: HEPE_F, Shape: (8591, 3)
Arquivo válido: DXXFEM_F, Shape: (8185, 20)
Arquivo válido: HEPBD_F, Shape: (8591, 4)
Arquivo válido: CDQ_F, Shape: (4135, 17)
Arquivo válido: DEET_F, Shape: (2831, 9)
Arquivo válido: DIQ_F, Shape: (10109, 20)
Arquivo válido: DEQ_F, Shape: (4145, 9)
Arquivo válido: HEPA_F, Shape: (9522, 2)
Arq

/var/folders/q3/ycc4hmlj3493q0x11zn3vgcc0000gn/T/ipykernel_4164/2045380523.py:313: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined[flag_name] = flag_values.astype(int)
/var/folders/q3/ycc4hmlj3493q0x11zn3vgcc0000gn/T/ipykernel_4164/2045380523.py:313: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_combined[flag_name] = flag_values.astype(int)
/var/folders/q3/ycc4hmlj3493q0x11zn3vgcc0000gn/T/ipykernel_4164/2045380523.py:313: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra


Coluna: MCQ_MCQ370A
Total de valores imputados: 5050
Valores imputados na classe 0: 4013
Valores imputados na classe 1: 1037
Porcentagem na classe 1: 20.53%

Estatísticas dos valores:
Originais:
count    2874.00
mean        1.35
std         0.52
min         1.00
25%         1.00
50%         1.00
75%         2.00
max         9.00
Name: MCQ_MCQ370A, dtype: float64

Imputados:
count    5050.00
mean        1.34
std         0.28
min         1.00
25%         1.00
50%         1.33
75%         1.67
max         4.00
Name: MCQ_MCQ370A, dtype: float64

Coluna: INQ_INQ020
Total de valores imputados: 18
Valores imputados na classe 0: 13
Valores imputados na classe 1: 5
Porcentagem na classe 1: 27.78%

Estatísticas dos valores:
Originais:
count    7906.00
mean        1.41
std         0.69
min         1.00
25%         1.00
50%         1.00
75%         2.00
max         9.00
Name: INQ_INQ020, dtype: float64

Imputados:
count    18.00
mean      1.33
std       0.30
min       1.00
25%       1.00
50%     

In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.base import BaseEstimator, ClassifierMixin

class StackingEnsembleCV(BaseEstimator, ClassifierMixin):
    def __init__(self, n_features=40, n_folds=5):
        self.n_folds = n_folds
        # Base models (first level)
        self.base_models = [
            ('gb', GradientBoostingClassifier(
                n_estimators=170,
                learning_rate=0.15,
                random_state=42,
                min_samples_split=5,
                min_samples_leaf=5
            )),
            ('rf', RandomForestClassifier(
                n_estimators=200,
                max_depth=10,
                min_samples_split=4,
                min_samples_leaf=4,
                random_state=42
            )),
            ('xgb', XGBClassifier(
                n_estimators=150,
                learning_rate=0.12,
                max_depth=6,
                random_state=42,
                eval_metric='logloss'
            )),
            ('lgbm', LGBMClassifier(
                n_estimators=150,
                learning_rate=0.12,
                max_depth=6,
                random_state=42
            ))
        ]
        
        # Meta-model (second level)
        self.meta_model = LogisticRegression(C=0.5, max_iter=800, penalty='l2', random_state=42)
        self.n_features = n_features
        self.classes_ = np.array([0, 1])
        
    def fit(self, X, y):
        self.classes_ = np.unique(y)
        
        # Initialize cross-validation
        kf = KFold(n_splits=self.n_folds, shuffle=True, random_state=42)
        
        # Initialize array for meta features
        meta_features = np.zeros((X.shape[0], len(self.base_models)))
        
        # For each base model
        for i, (name, model) in enumerate(self.base_models):
            # For each fold
            for train_idx, val_idx in kf.split(X):
                X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
                y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
                
                # Apply SMOTE only to training fold
                smote = SMOTE(random_state=42)
                X_train_resampled, y_train_resampled = smote.fit_resample(X_train_fold, y_train_fold)
                
                # Train model on the fold
                weights = np.where(y_train_resampled == 1, 5, 1)
                model.fit(X_train_resampled, y_train_resampled,
                         sample_weight=weights if name in ['gb', 'rf'] else None)
                
                # Generate predictions for validation fold
                meta_features[val_idx, i] = model.predict_proba(X_val_fold)[:, 1]
        
        # Train meta-model on the full meta-features
        self.meta_model.fit(meta_features, y)
        
        # Store base models trained on full dataset
        self.final_base_models = []
        for name, model in self.base_models:
            self.final_base_models.append((name, model))
        
        return self
    
    def predict_proba(self, X):
        meta_features = self.get_meta_features(X)
        return self.meta_model.predict_proba(meta_features)
    
    def predict(self, X):
        return self.meta_model.predict(self.get_meta_features(X))
    
    def get_meta_features(self, X):
        meta_features = np.zeros((X.shape[0], len(self.base_models)))
        for i, (name, model) in enumerate(self.base_models):
            meta_features[:, i] = model.predict_proba(X)[:, 1]
        return meta_features
    
    def decision_function(self, X):
        """Required for ROC AUC scoring"""
        return self.predict_proba(X)[:, 1]

def train_stacking_model_cv(df, target_cols, n_folds=5):
    """
    Train the stacking ensemble model with cross-validation.
    """
    # First split into train and test to have a true holdout set
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    
    # Encode categorical variables
    le = LabelEncoder()
    categorical_columns = train_df.select_dtypes(include=['object']).columns
    
    # Fit encoder on training data only
    train_encoded = train_df.copy()
    test_encoded = test_df.copy()
    
    for column in categorical_columns:
        le = LabelEncoder()
        train_encoded[column] = le.fit_transform(train_encoded[column].astype(str))
        # Transform test data with training encoders
        test_encoded[column] = le.transform(test_encoded[column].astype(str))

    # Prepare features and target
    X_train = train_encoded.drop(columns=target_cols, errors='ignore')
    y_train = train_encoded['target'].astype(int)
    X_test = test_encoded.drop(columns=target_cols, errors='ignore')
    y_test = test_encoded['target'].astype(int)

    # Initialize cross-validation
    cv = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    # Initialize and train stacking ensemble
    stacking = StackingEnsembleCV(n_folds=n_folds)
    
    # Perform cross-validation on training data
    cv_scores = cross_val_score(stacking, X_train, y_train, cv=cv, scoring='roc_auc')
    
    # Train final model on full training dataset
    stacking.fit(X_train, y_train)
    
    # Generate predictions for the test dataset
    predictions = stacking.predict(X_test)
    probabilities = stacking.predict_proba(X_test)[:, 1]

    # Generate evaluation metrics
    results = {
        'model': stacking,
        'cv_scores': cv_scores,
        'cv_mean_score': np.mean(cv_scores),
        'cv_std_score': np.std(cv_scores),
        'confusion_matrix': confusion_matrix(y_test, predictions),
        'classification_report': classification_report(y_test, predictions),
        'roc_auc': roc_auc_score(y_test, probabilities),
        'predictions': predictions,
        'probabilities': probabilities,
        'y_true': y_test
    }

    return results

# Example usage
target_columns = ["target", "BMI_percentil", "T_score_fn", "RIDAGEYR_percentil",
                 "BMXWT_percentil", "BMXHT_percentil", "BMXARMC_percentil",
                 "T_score_tf", "DXXT8_DXXPOSTH", "DXXT4_DXXANTEH", "DXXT7_DXXPT16Y",
                 "DXXFEM_DXAFMRD0", "T_score_sp", "DXXSPN_DXXOSBMD",
                 "DXXFEM_DXAFMRD0", "DXXFEM_DXXNKBMD", "DXXFEM_DXXOFBMD"]

results_stacking_cv = train_stacking_model_cv(df_final, target_columns, n_folds=5)

print("\nCross-validation scores:")
for i, score in enumerate(results_stacking_cv['cv_scores'], 1):
    print(f"Fold {i}: {score:.4f}")
print(f"\nMean CV Score: {results_stacking_cv['cv_mean_score']:.4f} (+/- {results_stacking_cv['cv_std_score']*2:.4f})")
print("\nFinal Model Classification Report:")
print(results_stacking_cv['classification_report'])
print(f"\nFinal Model ROC AUC Score: {results_stacking_cv['roc_auc']:.4f}")


Cross-validation scores:
Fold 1: 0.9461
Fold 2: 0.9365
Fold 3: 0.9271
Fold 4: 0.9016
Fold 5: 0.9322

Mean CV Score: 0.9287 (+/- 0.0298)

Final Model Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.95      1258
           1       0.90      0.72      0.80       327

    accuracy                           0.93      1585
   macro avg       0.92      0.85      0.88      1585
weighted avg       0.92      0.93      0.92      1585


Final Model ROC AUC Score: 0.9377


In [ ]:
# save_model.py
import joblib
from pathlib import Path

def save_trained_model(model, output_dir="models"):
    """
    Salva o modelo treinado e seus componentes
    """
    # Criar diretório se não existir
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    
    # Salvar o modelo completo
    model_path = Path(output_dir) / "stacking_ensemble_model.joblib"
    joblib.dump(model, model_path)
    print(f"Modelo salvo em: {model_path}")

# Uso
if __name__ == "__main__":
    # Assumindo que results_stacking_cv é o resultado do seu treinamento
    model = results_stacking_cv['model']
    save_trained_model(model)